<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/Project/Administration/administration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/CloudComputing-tirgul/Project/Index/
%run index.ipynb

In [ ]:
index = get_index_from_db()

In [ ]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate("/content/CloudComputing-tirgul/Project/Json/sak.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
# Define the function to update or add a document
def update_or_add_statistic(statistic_name):
    statistics_ref = db.collection("statisticsPanthers").document(statistic_name)
    doc = statistics_ref.get()
    if doc.exists:
        # Document exists, update the counter
        new_value = doc.to_dict().get("counter", 0) + 1
        statistics_ref.update({"counter": new_value})
    else:
        # Document doesn't exist, create a new one
        statistics_ref.set({"counter": 1})

In [ ]:
# Function to get indexes and populate a combo box in a notebook.
def get_indexes_keys():
  index = get_index_from_db(db)
  indexes = list(index.keys())
  indexesArray = []
  for idx in indexes:
      indexesArray.append(idx)
  # Convert Python arrays to JavaScript arrays
  indexesArrayJS = ', '.join(f'"{indx}"' for indx in indexes)

  js_code = f"""
    <script>
      populateFirstCombo([ {indexesArrayJS} ]);
    </script>
    """
  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.get_indexes_keys', get_indexes_keys)

# Function to get values for selected index and populate a second combo box.
def get_indexes_values(indexes):
    index = get_index_from_db(db)
    url = list(index[indexes].keys())
    # Convert Python arrays to JavaScript arrays
    urlArray = ', '.join(f'"{val}"' for val in url)
    js_code = f"""
      <script>
        populateSecondCombo([ {urlArray} ]);
      </script>
      """
    display(HTML(js_code))
# Register the function to be called from JavaScript
output.register_callback('notebook.get_indexes_values', get_indexes_values)

In [ ]:
# Generates random indexes for the statistics page.
def get_random_indexes_with_counters():
    # Randomly select 5 indexes
    random_indexes = random.sample(list(index.keys()), 5)

    indexes = []
    counters = []

    # Calculate sum of counters for each selected index
    for idx in random_indexes:
        index_sum = sum(entry['counter'] for entry in index[idx].values())
        indexes.append(idx)
        counters.append(index_sum)

     # Convert Python arrays to JavaScript arrays
    indexesArray = ', '.join(f'"{indx}"' for indx in indexes)
    countersArray = ', '.join(f'"{counter}"' for counter in counters)

    js_code = f"""
    <script>
      generateGraph([ {indexesArray} ], [ {countersArray} ]);
    </script>
    """
    display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.get_random_indexes_with_counters', get_random_indexes_with_counters)

In [ ]:
# Function to get key with minimum counter
def get_key_with_min_counter():
    statistics_ref = db.collection("statisticsPanthers")

    query = statistics_ref.order_by("counter")

    # Get the first document (minimum counter)
    min_doc = query.limit(1).get()

    for doc in min_doc:
        min_key = doc.id
        min_counter = doc.to_dict()["counter"]

    return min_key, min_counter

# Function to get key with maximum counter
def get_key_with_max_counter():
    statistics_ref = db.collection("statisticsPanthers")

    query = statistics_ref.order_by("counter", direction=firestore.Query.DESCENDING)

    # Get the first document (maximum counter)
    max_doc = query.limit(1).get()

    for doc in max_doc:
        max_key = doc.id
        max_counter = doc.to_dict()["counter"]

    return max_key, max_counter

# Function to get keys with min and max counters
def get_min_max_keys():
    min_key, min_counter = get_key_with_min_counter()
    max_key, max_counter = get_key_with_max_counter()
    # Convert counters to strings
    min_counter_str = str(min_counter)
    max_counter_str = str(max_counter)
    js_code =  f"""
    <script>
      updateStatisticsBlock("{min_key}", "{min_counter_str}", "{max_key}", "{max_counter_str}");
    </script>
    """
    display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.get_min_max_keys', get_min_max_keys)

In [ ]:
# Removes the index from the database.
def delete_index_from_db(index_to_delete):
    collection_ref = db.collection("PantherIndex")
    docs = collection_ref.get()
    for doc in docs:
        # Searches the index to delete from the received list of documents.
        if doc.to_dict()['term'] == index_to_delete:
            doc.reference.delete()
            index = get_index_from_db(db)
            break
output.register_callback('notebook.delete_index_from_db', delete_index_from_db)

In [ ]:
# Removes the link from the database.
def delete_link_from_db(link_index, link_to_delete):
    collection_ref = db.collection("PantherIndex")
    docs = collection_ref.get()
    for doc in docs:
        doc_dict = doc.to_dict()
         # Searches the link to delete from the received list of documents.
        if "DocId" in doc_dict:
            doc_id_links = doc_dict["DocId"]
            if link_to_delete in doc_id_links:
                doc_id_links.pop(link_to_delete)
                doc.reference.update({"DocId": doc_id_links})
                index = get_index_from_db(db)
                break
output.register_callback('notebook.delete_link_from_db', delete_link_from_db)